In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('movie_data.csv')

In [3]:
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [4]:
sentiments = df['sentiment']
reviews = df['review']

In [5]:
total_len = len(sentiments)

In [6]:
train_size = int(total_len*0.8)
test_size = total_len-train_size

print('Train size=',train_size)
print('Test size=',test_size)

Train size= 40000
Test size= 10000


In [7]:
train_x = df.loc[:train_size, 'review'].values
train_y = df.loc[:train_size, 'sentiment'].values

test_x = df.loc[train_size:total_len, 'review'].values
test_y = df.loc[train_size:total_len, 'sentiment'].values


In [8]:
print(type(sentiments))

<class 'pandas.core.series.Series'>


In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


In [ ]:
reviews = df['review'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)

In [ ]:
max_length = max([ len(s.split()) for s in reviews])

In [ ]:
print(max_length)

2470


In [ ]:
vocab_size = len(tokenizer.word_index)+1
X_train_tokens = tokenizer.texts_to_sequences(train_x)
X_test_tokens = tokenizer.texts_to_sequences(test_x)


X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Embedding

Using TensorFlow backend.


In [ ]:
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2470, 100)         12425300  
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 12,438,101
Trainable params: 12,438,101
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print('Train')
model.fit(X_train_pad, train_y, epochs = 25, batch_size=128, validation_data=(X_test_pad, test_y), verbose=2)

Train
Train on 40001 samples, validate on 10000 samples
Epoch 1/25
